# Table of Contents
 <p><div class="lev2"><a href="#Ładowanie-przykładowych-zestawów-danych"><span class="toc-item-num">0.1&nbsp;&nbsp;</span>Ładowanie przykładowych zestawów danych</a></div><div class="lev2"><a href="#Podstawowa-ocena-na-podstawie-kroswalidacji"><span class="toc-item-num">0.2&nbsp;&nbsp;</span>Podstawowa ocena na podstawie kroswalidacji</a></div><div class="lev2"><a href="#Grid-testing"><span class="toc-item-num">0.3&nbsp;&nbsp;</span>Grid testing</a></div><div class="lev3"><a href="#Dataset-Iris"><span class="toc-item-num">0.3.1&nbsp;&nbsp;</span>Dataset Iris</a></div><div class="lev4"><a href="#Trochę-czystsza-prezentacja"><span class="toc-item-num">0.3.1.1&nbsp;&nbsp;</span>Trochę czystsza prezentacja</a></div><div class="lev3"><a href="#Dataset-Digits"><span class="toc-item-num">0.3.2&nbsp;&nbsp;</span>Dataset Digits</a></div><div class="lev2"><a href="#Testowanie-kilku-klasyfikatorów"><span class="toc-item-num">0.4&nbsp;&nbsp;</span>Testowanie kilku klasyfikatorów</a></div><div class="lev3"><a href="#Dataset-Iris"><span class="toc-item-num">0.4.1&nbsp;&nbsp;</span>Dataset Iris</a></div><div class="lev4"><a href="#Parametery-najlepszego-klasyfikatora"><span class="toc-item-num">0.4.1.1&nbsp;&nbsp;</span>Parametery najlepszego klasyfikatora</a></div><div class="lev3"><a href="#Dataset-Digits"><span class="toc-item-num">0.4.2&nbsp;&nbsp;</span>Dataset Digits</a></div><div class="lev4"><a href="#Parametry-najlepszego-klasyfikatora"><span class="toc-item-num">0.4.2.1&nbsp;&nbsp;</span>Parametry najlepszego klasyfikatora</a></div><div class="lev3"><a href="#Porównanie-parametrów-klasyfikatorów-dla-obu-zestawów-danych"><span class="toc-item-num">0.4.3&nbsp;&nbsp;</span>Porównanie parametrów klasyfikatorów dla obu zestawów danych</a></div><div class="lev3"><a href="#Classification-report"><span class="toc-item-num">0.4.4&nbsp;&nbsp;</span>Classification report</a></div><div class="lev2"><a href="#Przykładowe-wykorzystanie-wyuczonego-klasyfikatora"><span class="toc-item-num">0.5&nbsp;&nbsp;</span>Przykładowe wykorzystanie wyuczonego klasyfikatora</a></div>

In [1]:
import numpy as np
import pandas as pd
from scipy.stats import randint
from operator import itemgetter
import sklearn.datasets as datasets
from sklearn import svm, grid_search, datasets, cross_validation, metrics, ensemble, neighbors, linear_model
from sklearn.ensemble import ExtraTreesClassifier
from datetime import datetime

## Ładowanie przykładowych zestawów danych

In [2]:
iris_dataset = datasets.load_iris()
digits_dataset = datasets.load_digits(10)

In [3]:
print(iris_dataset['DESCR'])

Iris Plants Database

Notes
-----
Data Set Characteristics:
    :Number of Instances: 150 (50 in each of three classes)
    :Number of Attributes: 4 numeric, predictive attributes and the class
    :Attribute Information:
        - sepal length in cm
        - sepal width in cm
        - petal length in cm
        - petal width in cm
        - class:
                - Iris-Setosa
                - Iris-Versicolour
                - Iris-Virginica
    :Summary Statistics:

    ============== ==== ==== ======= ===== ====================
                    Min  Max   Mean    SD   Class Correlation
    ============== ==== ==== ======= ===== ====================
    sepal length:   4.3  7.9   5.84   0.83    0.7826
    sepal width:    2.0  4.4   3.05   0.43   -0.4194
    petal length:   1.0  6.9   3.76   1.76    0.9490  (high!)
    petal width:    0.1  2.5   1.20  0.76     0.9565  (high!)
    ============== ==== ==== ======= ===== ====================

    :Missing Attribute Values: None
  

## Podstawowa ocena na podstawie kroswalidacji

Poprzez testowanie na innych danych niż było przeprowadzone uczenie, kroswalidacja pozwala nam na otrzymanie dość wiarygodnej oceny danego klasyfikatora.
W bibliotece scikit-learn wykonanie kroswalidacji jest bardzo proste, wystarczy do metody przekazać klasyfikator, który chcemy sprawdzić oraz dane i odpowiadające im klasy.  
[sikit-learn docs](http://scikit-learn.org/stable/modules/cross_validation.html)  
[wikipedia](https://pl.wikipedia.org/wiki/Sprawdzian_krzy%C5%BCowy)

In [4]:
svc = svm.SVC()
linear_svc = svm.LinearSVC()

In [5]:
cross_validation.cross_val_score(svc, iris_dataset['data'], iris_dataset['target']).mean()

0.97344771241830064

In [6]:
cross_validation.cross_val_score(linear_svc, iris_dataset['data'], iris_dataset['target']).mean()

0.96691176470588225

## Grid testing

Inicjalizujemy klasyfikator SVM  

**SVM** - Support Vector Machine:
- **SVC** - Support Vector Classification
- **SVR** - Support Vector Regressor
    
[scikit-learn docs](http://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html)

In [7]:
svc = svm.SVC()

Słownik parametrów do przetestowania. Grid Search przyjmuję słownik lub listę słowników parametrów, które należy przekazać do konstruktora klasyfikatora. Z każdego słownika tworzy macierz kombinacji tych parametrów.

In [8]:
parameters = [{'kernel':['rbf'], 'C':[1e-4, 1e-3, 1e-2, 1e0, 1e1, 1e2, 1e3]},
              {'kernel':['poly'], 'C':[1e-4, 1e-3, 1e-2, 1e0, 1e1, 1e2, 1e3], 'degree':[2,3]},
             {'kernel':['linear'], 'C':[1e-4, 1e-3, 1e-2, 1e0, 1e1, 1e2, 1e3]}]

### Dataset Iris

Sprawdźmy, które parametry najlepiej się sprawdzają w połączeniu klasyfikatora SVM i zestawu Iris

Po wywołaniu metody **fit(X,y)** na obiekcie GridSearchCV,  wykonywana jest kroswalidacja na instancjach klasyfikatora SVM z różnymi kombinacjami parametrów. Po otrzymaniu wyników z każdej kroswalidacji, wybierany jest najlepszy zestaw parametrów.

Kroswalidacja zwraca wyniki otrzymane z domyślnej metody **score(X,y)** zaimplementowanej w klasyfikatorze. Możemy przekazać inny *scorer*, na podstawie którego ma być robiona ocena  
[Model evaluation - scikit-learn doc](http://scikit-learn.org/stable/modules/model_evaluation.html)

In [9]:
gs = grid_search.GridSearchCV(svc, parameters, n_jobs=4)
gs.fit(iris_dataset['data'], iris_dataset['target'])

GridSearchCV(cv=None, error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
       fit_params={}, iid=True, n_jobs=4,
       param_grid=[{'kernel': ['rbf'], 'C': [0.0001, 0.001, 0.01, 1.0, 10.0, 100.0, 1000.0]}, {'kernel': ['poly'], 'degree': [2, 3], 'C': [0.0001, 0.001, 0.01, 1.0, 10.0, 100.0, 1000.0]}, {'kernel': ['linear'], 'C': [0.0001, 0.001, 0.01, 1.0, 10.0, 100.0, 1000.0]}],
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)

Zestaw najlepszych parametrów

In [10]:
gs.best_params_

{'C': 10.0, 'kernel': 'rbf'}

Wyniki dla wszystkich zestawów parametrów

In [11]:
gs.grid_scores_

[mean: 0.93333, std: 0.03277, params: {'kernel': 'rbf', 'C': 0.0001},
 mean: 0.93333, std: 0.03277, params: {'kernel': 'rbf', 'C': 0.001},
 mean: 0.93333, std: 0.03277, params: {'kernel': 'rbf', 'C': 0.01},
 mean: 0.97333, std: 0.00897, params: {'kernel': 'rbf', 'C': 1.0},
 mean: 0.98000, std: 0.01601, params: {'kernel': 'rbf', 'C': 10.0},
 mean: 0.96000, std: 0.04217, params: {'kernel': 'rbf', 'C': 100.0},
 mean: 0.95333, std: 0.03669, params: {'kernel': 'rbf', 'C': 1000.0},
 mean: 0.86000, std: 0.00404, params: {'kernel': 'poly', 'degree': 2, 'C': 0.0001},
 mean: 0.87333, std: 0.00809, params: {'kernel': 'poly', 'degree': 3, 'C': 0.0001},
 mean: 0.88667, std: 0.02253, params: {'kernel': 'poly', 'degree': 2, 'C': 0.001},
 mean: 0.96667, std: 0.01820, params: {'kernel': 'poly', 'degree': 3, 'C': 0.001},
 mean: 0.95333, std: 0.02402, params: {'kernel': 'poly', 'degree': 2, 'C': 0.01},
 mean: 0.97333, std: 0.02446, params: {'kernel': 'poly', 'degree': 3, 'C': 0.01},
 mean: 0.96667, std: 

#### Trochę czystsza prezentacja

In [12]:
# source: http://scikit-learn.org/stable/auto_examples/model_selection/randomized_search.html
# Utility function to report best scores
def report(grid_scores, n_top=3):
    top_scores = sorted(grid_scores, key=itemgetter(1), reverse=True)[:n_top]
    for i, score in enumerate(top_scores):
        print("Model with rank: {0}".format(i + 1))
        print("Mean validation score: {0:.5f} (std: {1:.5f})".format(
              score.mean_validation_score,
              np.std(score.cv_validation_scores)))
        print("Parameters: {0}".format(score.parameters))
        print("")

In [13]:
report(gs.grid_scores_, 5)

Model with rank: 1
Mean validation score: 0.98000 (std: 0.01601)
Parameters: {'kernel': 'rbf', 'C': 10.0}

Model with rank: 2
Mean validation score: 0.98000 (std: 0.01602)
Parameters: {'kernel': 'linear', 'C': 1.0}

Model with rank: 3
Mean validation score: 0.97333 (std: 0.00897)
Parameters: {'kernel': 'rbf', 'C': 1.0}

Model with rank: 4
Mean validation score: 0.97333 (std: 0.02446)
Parameters: {'kernel': 'poly', 'degree': 3, 'C': 0.01}

Model with rank: 5
Mean validation score: 0.97333 (std: 0.03697)
Parameters: {'kernel': 'linear', 'C': 10.0}



### Dataset Digits

Dokładnie to samo dla zestawu danych Digits

In [14]:
gs = grid_search.GridSearchCV(svc, parameters, n_jobs=4)
gs.fit(digits_dataset['data'], digits_dataset['target'])

GridSearchCV(cv=None, error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
       fit_params={}, iid=True, n_jobs=4,
       param_grid=[{'kernel': ['rbf'], 'C': [0.0001, 0.001, 0.01, 1.0, 10.0, 100.0, 1000.0]}, {'kernel': ['poly'], 'degree': [2, 3], 'C': [0.0001, 0.001, 0.01, 1.0, 10.0, 100.0, 1000.0]}, {'kernel': ['linear'], 'C': [0.0001, 0.001, 0.01, 1.0, 10.0, 100.0, 1000.0]}],
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)

In [15]:
report(gs.grid_scores_, 5)

Model with rank: 1
Mean validation score: 0.96049 (std: 0.01269)
Parameters: {'kernel': 'poly', 'degree': 3, 'C': 0.0001}

Model with rank: 2
Mean validation score: 0.96049 (std: 0.01269)
Parameters: {'kernel': 'poly', 'degree': 3, 'C': 0.001}

Model with rank: 3
Mean validation score: 0.96049 (std: 0.01269)
Parameters: {'kernel': 'poly', 'degree': 3, 'C': 0.01}

Model with rank: 4
Mean validation score: 0.96049 (std: 0.01269)
Parameters: {'kernel': 'poly', 'degree': 3, 'C': 1.0}

Model with rank: 5
Mean validation score: 0.96049 (std: 0.01269)
Parameters: {'kernel': 'poly', 'degree': 3, 'C': 10.0}



## Testowanie kilku klasyfikatorów

Wcześniej przetestowaliśmy jeden klasyfikator z różnymi parametrami.  
Dzięki temu dowiedzieliśmy się jaki najlepszy wynik można osiągnąć z danym klasyfikatorem. Sprawdzając w ten sposób kilka różnych klasyfikatorów, bardzo łatwo jest wybrać najlepszy z nich (przynajmniej pod względem danej metody oceniania - _scoring_)

In [16]:
def test_classifiers(classifiers_to_test, X, y):
    """
    Zwróć listę przekazanych klasyfikatorów z najlepszymi wynikami osiągnietymi dla zestawu danych X,y
    
    Args:
        classifiers_to_test: lista klasyfikatorów do sprawdzenia, 
                             składa się z słowników z wartościami: name, classifier, parameters
        X: dane wejściowe do klasyfikacji
        y: klasy odpowiadające danym wejściowym
    """
    classifiers_results = []
    for classifier in classifiers_to_test:  # iterujemy po każdym słowniku z informacjami o klasyfikatorze
        start = datetime.now()
        gs = grid_search.GridSearchCV(classifier['classifier'],  # inicjalizujemy GridSearchCV
                                      classifier['parameters'], 
                                      n_jobs=4,  # GridSearchCV będzie działał na 4 wątkach
                                      scoring='accuracy')  # ustawiamy wszystkim klasyfikatorom na sztywno rodzaj _scoringu_
        gs.fit(X, y)
        classifiers_results.append({'name': classifier['name'], 
                                         'best_score': gs.best_score_, 
                                         'grid_search': gs})  # zapisujemy wyniki

        # na bierząco wyświetlamy wyniki
        print("{0}\nscore: {1}\ntime: {2}\n*=======================*".
              format(classifier['name'], 
                     gs.best_score_, 
                     (datetime.now() - start))) 
        
    return classifiers_results

Tworzymy listę klasyfikatorów z odpowiednimi zestawami parametrów. W celu czytelniejszej prezentacji przekazujemy też nazwę klasyfikatora.

In [17]:
classifiers_to_test = [
    {
        'name': 'SVC',
        'classifier': svm.SVC(),
        'parameters': [{'kernel':['rbf'], 'C':[1e-4, 1e-3, 1e-2, 1e0, 1e1, 1e2, 1e3]},
                       {'kernel':['linear'], 'C':[1e-4, 1e-3, 1e-2, 1e0, 1e1, 1e2, 1e3]}]
    },
    {
        'name': 'LinearSVC',
        'classifier': svm.LinearSVC(),
        'parameters': [{'max_iter':[1e3, 1e4, 1e5], 'C':[1e-4, 1e-3, 1e-2, 1e0, 1e1, 1e2, 1e3]}]
    },
    {
        'name': 'ExtraTreesClassifier',
        'classifier': ensemble.ExtraTreesClassifier(),
        'parameters': [{'n_estimators':[5,10,20,50], 'min_samples_leaf':[1,2,4,8]}]
    },
    {
        'name': 'RandomForestClassifier',
        'classifier': ensemble.RandomForestClassifier(),
        'parameters': [{'n_estimators':[5,10,20,50], 'min_samples_leaf':[1,2,4,8]}]
    },
    {
        'name': 'KNeighborsClassifier',
        'classifier': neighbors.KNeighborsClassifier(),
        'parameters': [{'n_neighbors':[5,10,20], 'p':[1,2]}]
    },
    {
        'name': 'LogisticRegression',
        'classifier': linear_model.LogisticRegression(),
        'parameters': [{'max_iter':[1e3, 1e4, 1e5], 'C':[1e-4, 1e-3, 1e-2, 1e0, 1e1, 1e2, 1e3]}]
    }
]

### Dataset Iris

In [18]:
X, y = iris_dataset['data'], iris_dataset['target']
classifiers_results_iris = test_classifiers(classifiers_to_test, X , y)

SVC
score: 0.98
time: 0:00:00.186209
*=======================*
LinearSVC
score: 0.9666666666666667
time: 0:00:03.819588
*=======================*
ExtraTreesClassifier
score: 0.9666666666666667
time: 0:00:00.887043
*=======================*
RandomForestClassifier
score: 0.9733333333333334
time: 0:00:00.940280
*=======================*
KNeighborsClassifier
score: 0.9866666666666667
time: 0:00:00.160544
*=======================*
LogisticRegression
score: 0.9733333333333334
time: 0:00:00.281516
*=======================*


#### Parametery najlepszego klasyfikatora

Sprawdzamy, który klasyfikator najlepiej wypadł i jakie miał parametry

In [19]:
classifiers_results_iris.sort(key=itemgetter('best_score'), reverse=True)  # sortujemy po najlepszym wyniku
print(classifiers_results_iris[0]['name'])
print(classifiers_results_iris[0]['grid_search'].best_params_)

KNeighborsClassifier
{'p': 2, 'n_neighbors': 5}


Ranking parametrów najlepszego klasyfikatora

In [20]:
report(classifiers_results_iris[0]['grid_search'].grid_scores_)

Model with rank: 1
Mean validation score: 0.98667 (std: 0.00924)
Parameters: {'p': 2, 'n_neighbors': 5}

Model with rank: 2
Mean validation score: 0.97333 (std: 0.00897)
Parameters: {'p': 1, 'n_neighbors': 5}

Model with rank: 3
Mean validation score: 0.97333 (std: 0.00897)
Parameters: {'p': 1, 'n_neighbors': 10}



### Dataset Digits

In [21]:
X, y = digits_dataset['data'], digits_dataset['target']
classifiers_results_digits = test_classifiers(classifiers_to_test, X , y)

SVC
score: 0.9515859766277128
time: 0:00:04.169793
*=======================*
LinearSVC
score: 0.9298831385642737
time: 0:00:17.975507
*=======================*
ExtraTreesClassifier
score: 0.9465776293823038
time: 0:00:01.324307
*=======================*
RandomForestClassifier
score: 0.9376739009460211
time: 0:00:01.612046
*=======================*
KNeighborsClassifier
score: 0.9627156371730662
time: 0:00:01.079045
*=======================*
LogisticRegression
score: 0.9298831385642737
time: 0:00:06.140566
*=======================*


#### Parametry najlepszego klasyfikatora

In [22]:
classifiers_results_digits.sort(key=itemgetter('best_score'), reverse=True)
print(classifiers_results_digits[0]['name'])
print(classifiers_results_digits[0]['grid_search'].best_params_)

KNeighborsClassifier
{'p': 2, 'n_neighbors': 5}


### Porównanie parametrów klasyfikatorów dla obu zestawów danych

Tworzymy **pandas.DataFrame** z wyników obu testów. Następnie łączymy je poziomo, dzięki czemu mamy dobre porównanie klasyfikatorów

In [23]:
iris_results_df = pd.DataFrame([[result['name'], result['best_score']] 
                                for result in classifiers_results_iris],
                               columns=['Iris', 'best_score'])
digits_results_df = pd.DataFrame([[result['name'], result['best_score']] 
                                  for result in classifiers_results_digits],
                                 columns=['Digits', 'best_score'])
pd.concat([iris_results_df, digits_results_df], axis=1)

,Iris,best_score,Digits,best_score
0,KNeighborsClassifier,0.986667,KNeighborsClassifier,0.962716
1,SVC,0.980000,SVC,0.951586
2,RandomForestClassifier,0.973333,ExtraTreesClassifier,0.946578
3,LogisticRegression,0.973333,RandomForestClassifier,0.937674
4,LinearSVC,0.966667,LinearSVC,0.929883
5,ExtraTreesClassifier,0.966667,LogisticRegression,0.929883


### Classification report

W przypadku klasyfikacji wielu kategorii warto sprawdzić, jaki jest wynik klasyfikacji każdej kategorii. W tym celu możemy skorzystać z gotowej metody **sklearn.metrics.classification_report(y_true, y_pred)**

In [24]:
X, y = digits_dataset['data'], digits_dataset['target']

**GridSearchCV** po wybraniu najlepszych parametrów klasyfikatora, przeucza go na całym zestawie danych i sam zaczyna działać jak klasyfikator.  
Tzn. Wywołanie metory **predict(X)** na obiekcie **GridSearchCV**, zwraca wynik metody **predict(X)** na wybranym (najlepszym) klasyfikatorze (GridSearchCV.best_estimator)

In [25]:
y_pred = classifiers_results_digits[0]['grid_search'].predict(X) # Robimy predykcje na najlepszym klasyfikatorze dla zestawu digits

W pierwszej kolumnie są klasy (możemy przekazać nazwy klas do fukcji **classification_report**).  
Kolumny **precision**, **recall**, **f1-score** zawierają wyniki dla każdej klasy.  
Kolumna **support** to liczność danej klasy

In [26]:
print(metrics.classification_report(y, y_pred))

             precision    recall  f1-score   support

          0       1.00      1.00      1.00       178
          1       0.98      1.00      0.99       182
          2       1.00      1.00      1.00       177
          3       0.98      0.99      0.99       183
          4       0.99      1.00      1.00       181
          5       0.99      0.99      0.99       182
          6       0.99      1.00      0.99       181
          7       0.99      0.99      0.99       179
          8       0.99      0.98      0.98       174
          9       0.99      0.96      0.97       180

avg / total       0.99      0.99      0.99      1797



## Przykładowe wykorzystanie wyuczonego klasyfikatora

In [27]:
classifier = classifiers_results_iris[0]['grid_search']  # Klasyfikator dla zestawu iris

In [28]:
iris_dataset['data'][0]

array([ 5.1,  3.5,  1.4,  0.2])

In [29]:
iris_dataset['target'][0]

0

In [30]:
classifier.predict([iris_dataset['data'][0]])

array([0])